# Correr experimentos
En este archivo está el código para correr los experimentos y escribir los resultados en archivos CSV.
> Los archivos se guardan en la carpeta _resultados_.

In [1]:
import math, subprocess
import os
import pandas as pd
import numpy as np
from IPython.display import display, clear_output

A continuación leemos los datasets en dataframes de Pandas.

In [2]:
def leer_instancia(path_instancia):
    with open(path_instancia, "r") as f:
        return f.read();


df_grafos_propios = pd.read_csv("instancias/grafo-propio/indice.csv");
df_grafos_opt = pd.read_csv("instancias/descargas/indice.csv");
df_peor_caso_AGM = pd.read_csv("instancias/peor-caso-agm/indice.csv");
df_peor_caso_AG = pd.read_csv("instancias/peor-caso-ga/indice2.csv")
df_euleriano = pd.read_csv("instancias/grafo-euleriano/indice.csv")


In [3]:
display(df_grafos_opt)

,dataset,instancia,n,m,archivo,peso_optimo
0,sol_optimas,u1432,1432,1024596.0,instancias/descargas/data/u1432.tsp,152970
1,sol_optimas,gr666,666,221445.0,instancias/descargas/data/gr666.tsp,294358
2,sol_optimas,rl1304,1304,849556.0,instancias/descargas/data/rl1304.tsp,252948
3,sol_optimas,pr107,107,5671.0,instancias/descargas/data/pr107.tsp,44303
4,sol_optimas,rat99,99,4851.0,instancias/descargas/data/rat99.tsp,1211
...,...,...,...,...,...,...
81,sol_optimas,kroE100,100,4950.0,instancias/descargas/data/kroE100.tsp,22068
82,sol_optimas,att48,48,1128.0,instancias/descargas/data/att48.tsp,10628
83,sol_optimas,ulysses22,22,231.0,instancias/descargas/data/ulysses22.tsp,7013
84,sol_optimas,d493,493,121278.0,instancias/descargas/data/d493.tsp,35002


La siguiente función sirve para correr el código sobre una instancia ejecutando un método en particular.
Metodos:
- agm
- ins
- GA
- TABU_CICLOS
- TABU_ESTRUCTURA


In [4]:
def correr_experimento(metodo, archivo_instancia):
    # Leer archivo de la instancia.
    instancia = leer_instancia(archivo_instancia)
    argumentos = ["../TSP"]
    argumentos = argumentos + metodo.split(" ")
    display(argumentos[1])
    #print(instancia)
    
    
    # Crear proceso para ejecutar el codigo.
    process = subprocess.Popen(argumentos, stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    # Poner la instancia en la entrada estandar.
    process.stdin.write(instancia)
    process.stdin.flush()

    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code != 0: raise(F"Hubo un error en la experimentacion para el algoritmo: {algoritmo} con la instancia {archivo_instancia}.")
    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    tiempo_de_ejecucion = float(process.stderr.read());
    res = process.stdout.read().split(" ")
    tam = res[0] #tamaño del ciclo
    pyc =res[1].split("\n")
    peso = pyc[0] #peso del ciclo
    ciclo = pyc[1] #ciclo

    
    process.stdin.close();
    process.stdout.close();
    process.stderr.close();
    
    return [tiempo_de_ejecucion, tam, peso, ciclo]; #devuelvo un par de cosas extra por si las queremos usar

# Corremos los experimentos
Vamos a guardar una tabla con las ejecuciones y sus respectivos tiempos.

### Importante la celda de abajo para generar los experimentos!!

In [5]:
experimentos = [];
#algoritmos = ["ins", "agm", "GA",  "TABU_CICLOS", "TABU_ESTRUCTURA"]
algoritmos =["INS", "AGM", "GA"]; #de prueba para ver que funciona
algoritmos_tabu = ["TABU_CICLOS", "TABU_ESTRUCTURA"] # Para los experimentos de parametros
algoritmos_tabu_optimizado = ["TABU_CICLOS 160 30 300", "TABU_ESTRUCTURA 80 10 200"]

# armado de experimentos grafos euLeRianos

In [6]:
for n in range(0, df_euleriano.shape[0]):
    fila_n = df_euleriano.iloc[n];
    for algo in algoritmos:
        experimentos.append([fila_n["dataset"], fila_n["n"], fila_n["m"], algo, fila_n["archivo"]]);

# Ejecucion con grafos euLeRianos

In [7]:
columnas = ["dataset", "n", "m", "metodo", "tiempo", "costo"];
filas = [];
numero = 1
T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    costos  = []
    tiempos = []
    for i in range(0, T):
        resultado = correr_experimento(experimento[3], experimento[4])
        algoritmo = experimento[4]
        costos.append(resultado[2])
        tiempos.append(resultado[0]);
    costo = min(costos)
    tiempo = np.mean(tiempos);
    filas.append([experimento[0], experimento[1], experimento[2], experimento[3], tiempo,costo ]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultado_euler.csv", index=False, header=True);

'Experimento: 300/300'

'GA'

'GA'

'GA'

'GA'

'GA'

### Armado de experimentos grafos random

In [23]:
experimentos = []
for n in range(0, df_grafos_propios.shape[0]):
    fila_n = df_grafos_propios.iloc[n];
    for algo in algoritmos:
        experimentos.append([fila_n["dataset"], fila_n["n"], fila_n["m"], algo, fila_n["archivo"]]);
 

# Ejecucion de algoritmos con Grafos Random

In [24]:
columnas = ["dataset", "n", "m", "metodo", "tiempo", "costo"];
filas = [];
numero = 1
T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    costos  = []
    tiempos = []
    for i in range(0, T):
        resultado = correr_experimento(experimento[3], experimento[4])
        algoritmo = experimento[4]
        costos.append(resultado[2])
        tiempos.append(resultado[0]);
    costo = min(costos)
    tiempo = np.mean(tiempos);
    filas.append([experimento[0], experimento[1], experimento[2], experimento[3], tiempo,costo ]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultado_random.csv", index=False, header=True);

'Experimento: 300/300'

'GA'

'GA'

'GA'

'GA'

'GA'

# Ejecucion de los experimentos con optimos conocidos

In [5]:
df_grafos_opt.head()

,dataset,instancia,n,m,archivo,peso_optimo
0,sol_optimas,u1432,1432,1024596.0,instancias/descargas/data/u1432.tsp,152970
1,sol_optimas,gr666,666,221445.0,instancias/descargas/data/gr666.tsp,294358
2,sol_optimas,rl1304,1304,849556.0,instancias/descargas/data/rl1304.tsp,252948
3,sol_optimas,pr107,107,5671.0,instancias/descargas/data/pr107.tsp,44303
4,sol_optimas,rat99,99,4851.0,instancias/descargas/data/rat99.tsp,1211


In [6]:
experimentos_optimos = []


for n in range(0, df_grafos_opt.shape[0]):
    fila_n = df_grafos_opt.iloc[n];
    for algo in algoritmos:
        if (int(fila_n["n"])):
            experimentos_optimos.append([fila_n["dataset"], fila_n["n"], fila_n["m"], algo, fila_n["archivo"], fila_n["peso_optimo"]]);


In [8]:
columnas = ["dataset", "n", "m", "metodo", "tiempo", "costo", "gap"];
filas = [];
numero = 1
T = 3 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos_optimos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos_optimos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    costos  = []
    tiempos = []
    for i in range(0, T):
        resultado = correr_experimento(experimento[3], experimento[4])
        algoritmo = experimento[4]
        costos.append(resultado[2])
        tiempos.append(resultado[0]);
    costo = int(min(costos))
    tiempo = np.mean(tiempos);
    gap = (costo / int(experimento[5]) * 100) - 100# Lo que me falto para el costo optimo conocido  
    filas.append([experimento[0], experimento[1], experimento[2], experimento[3], tiempo,costo, gap]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultado_opts.csv", index=False, header=True);

'Experimento: 258/258'

'GA'

'GA'

'GA'

# ESTO NO LO CORRIMO MASTER

# Ejecucion del experimento de peor caso AGM

In [10]:
experimentos_AGM = []


for n in range(0, df_peor_caso_AGM.shape[0]):
    fila_n = df_peor_caso_AGM.iloc[n];
    for algo in algoritmos_tabu:
        experimentos_AGM.append([fila_n["dataset"], fila_n["n"], fila_n["m"], algo, fila_n["archivo"]]);

In [11]:
columnas = ["dataset", "n", "m", "metodo", "tiempo", "costo"];
filas = [];
numero = 1
T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos_AGM:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos_AGM)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    costos  = []
    tiempos = []
    for i in range(0, T):
        resultado = correr_experimento(experimento[3], experimento[4])
        algoritmo = experimento[4]
        costos.append(resultado[2])
        tiempos.append(resultado[0]);
    costo = min(costos)
    tiempo = np.mean(tiempos);
    filas.append([experimento[0], experimento[1], experimento[2], experimento[3], tiempo,costo]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultado_agm_tabu.csv", index=False, header=True);

'Experimento: 200/200'

'TABU_ESTRUCTURA'

'TABU_ESTRUCTURA'

'TABU_ESTRUCTURA'

'TABU_ESTRUCTURA'

'TABU_ESTRUCTURA'

# Y ACA SI CORRIMO

# Experimentacion de parametros TABU

## Con los grafos randoms

In [12]:
experimentos_TABU_random = []

fila_n = df_grafos_propios[df_grafos_propios["archivo"] == "instancias/grafo-propio/G-100.txt"]
fila_n = fila_n.iloc[0]
# como correr_experimentos le pasa de parametro a ./TSP TABU_CICLOS, agregamos aca como strings
# t memoria, vecindad, iteraciones
for algo in algoritmos_tabu:
    for t in range(10,101,20):
        for v in range(10,101,20):
            for it in range(100,501,100):
                algo_params = " " + str(t) + " " + str(v) + " " + str(it)
                experimentos_TABU_random.append([fila_n["dataset"], fila_n["n"], fila_n["m"], algo + algo_params, fila_n["archivo"], str(t), str(v), str(it)]);

In [13]:
print(len(experimentos_TABU_random))

1400


In [14]:
columnas = ["dataset", "n", "m", "metodo", "tiempo", "costo", "memoria", "vecindad", "iteraciones"];
filas = [];
numero = 1
T = 1 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos_TABU_random:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos_TABU_random)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    costos  = []
    tiempos = []
    for i in range(0, T):
        resultado = correr_experimento(experimento[3], experimento[4])
        costos.append(resultado[2])
        tiempos.append(resultado[0]);
    costo = min(costos)
    tiempo = np.mean(tiempos);
    t = experimento[5]
    v = experimento[6]
    it = experimento[7]
    filas.append([experimento[0], experimento[1], experimento[2], experimento[3], tiempo,costo, t, v, it]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultado_tabu_params_random.csv", index=False, header=True);

'Experimento: 1400/1400'

'TABU_ESTRUCTURA'

## Con los grafos de opt

In [27]:
df = df_grafos_opt.copy()

experimentos_TABU_opt = []
instancias = ["instancias/descargas/data/bier127.tsp", "instancias/descargas/data/d198.tsp", "instancias/descargas/data/a280.tsp", "instancias/descargas/data/lin318.tsp"
,"instancias/descargas/data/pr439.tsp", "instancias/descargas/data/tsp225.tsp", "instancias/descargas/data/pr299.tsp",
"instancias/descargas/data/rat195.tsp", "instancias/descargas/data/pr152.tsp",
"instancias/descargas/data/rd400.tsp"]


for instancia in instancias:
    fila_n = df[df["archivo"] == str(instancia)]
    fila_n = fila_n.iloc[0]
    

#for n in range(0, df_grafos_opts.shape[0]):
    #fila_n = df_grafos_opts.iloc[n];
    #if (fila_n["n"] < 400):
    
    for algo in algoritmos_tabu:
# como correr_experimentos le pasa de parametro a ./TSP TABU_CICLOS, agregamos aca como strings
# t memoria, vecindad, iteraciones
        for t in range(40,161,40):
            for v in range(10,71,20):
                for it in range(100,501,100):
                    algo_params = " " + str(t) + " " + str(v) + " " + str(it)
                    experimentos_TABU_opt.append([fila_n["dataset"], fila_n["n"], fila_n["m"], algo + algo_params, fila_n["archivo"], str(t), str(v), str(it)]);

In [28]:
columnas = ["dataset", "n", "m", "metodo", "tiempo", "costo", "memoria", "vecindad", "iteraciones"];
filas = [];
numero = 1
T = 1 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos_TABU_opt:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos_TABU_opt)))
    numero += 1
    #display(experimento)
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    costos  = []
    tiempos = []
    for i in range(0, T):
        resultado = correr_experimento(experimento[3], experimento[4])
        costos.append(resultado[2])
        tiempos.append(resultado[0]);
    costo = min(costos)
    tiempo = np.mean(tiempos);
    t = experimento[5]
    v = experimento[6]
    it = experimento[7]
    filas.append([experimento[0], experimento[1], experimento[2], experimento[3], tiempo,costo, t, v, it]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultado_tabu_params_opt.csv", index=False, header=True);

'Experimento: 1600/1600'

'TABU_ESTRUCTURA'

In [18]:
res = correr_experimento("TABU_CICLOS 50 50 100", "instancias/descargas/data/linhp318.tsp")
display(res)

'TABU_CICLOS'

[16777.0, '318', '48897', '1']

# experimentacion vecindad mas finoli

In [6]:
df = df_grafos_opt.copy()

experimentos_TABU_opt = []
instancias = ["instancias/descargas/data/bier127.tsp", "instancias/descargas/data/d198.tsp", "instancias/descargas/data/a280.tsp", "instancias/descargas/data/lin318.tsp"
,"instancias/descargas/data/pr439.tsp", "instancias/descargas/data/tsp225.tsp", "instancias/descargas/data/pr299.tsp",
"instancias/descargas/data/rat195.tsp", "instancias/descargas/data/pr152.tsp",
"instancias/descargas/data/rd400.tsp","instancias/descargas/data/u1060.tsp", "instancias/descargas/data/rat783.tsp", "instancias/descargas/data/st70.tsp","instancias/descargas/data/rat575.tsp","instancias/descargas/data/pr299.tsp" ]


for instancia in instancias:
    fila_n = df[df["archivo"] == str(instancia)]
    fila_n = fila_n.iloc[0]
    

#for n in range(0, df_grafos_opts.shape[0]):
    #fila_n = df_grafos_opts.iloc[n];
    #if (fila_n["n"] < 400):
    
    for algo in algoritmos_tabu:
# como correr_experimentos le pasa de parametro a ./TSP TABU_CICLOS, agregamos aca como strings
# t memoria, vecindad, iteraciones
        for v in range(5, 101, 5):
            algo_params = " " + str(160) + " " + str(v) + " " + str(300)
            experimentos_TABU_opt.append([fila_n["dataset"], fila_n["n"], fila_n["m"], algo + algo_params, fila_n["archivo"], str(160), str(v), str(300)]);

In [7]:
columnas = ["dataset", "n", "m", "metodo", "tiempo", "costo", "memoria", "vecindad", "iteraciones"];
filas = [];
numero = 1
T = 1 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos_TABU_opt:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos_TABU_opt)))
    numero += 1
    #display(experimento)
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    costos  = []
    tiempos = []
    for i in range(0, T):
        resultado = correr_experimento(experimento[3], experimento[4])
        costos.append(resultado[2])
        tiempos.append(resultado[0]);
    costo = min(costos)
    tiempo = np.mean(tiempos);
    t = experimento[5]
    v = experimento[6]
    it = experimento[7]
    filas.append([experimento[0], experimento[1], experimento[2], experimento[3], tiempo,costo, t, v, it]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultado_tabu_params_finoli_opt.csv", index=False, header=True);

'Experimento: 600/600'

'TABU_ESTRUCTURA'

# peor caso GA

In [22]:
experimentos = []
for n in range(0,  df_peor_caso_AG.shape[0]):
    fila_n = df_peor_caso_AG.iloc[n];
    for algo in algoritmos:
        experimentos.append([fila_n["dataset"], fila_n["n"], fila_n["m"], algo, fila_n["archivo"]]);

In [23]:
columnas = ["dataset", "n", "m", "metodo", "tiempo", "costo"];
filas = [];
numero = 1
T = 1 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    costos  = []
    tiempos = []
    for i in range(0, T):
        resultado = correr_experimento(experimento[3], experimento[4])
        algoritmo = experimento[3]
        costos.append(resultado[2])
        tiempos.append(resultado[0]);
    costo = min(costos)
    tiempo = np.mean(tiempos);
    filas.append([experimento[0], experimento[1], experimento[2], experimento[3], tiempo,costo]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultado_peor_ag2.csv", index=False, header=True);

'Experimento: 300/300'

'GA'

# Y ACA YA NO VITE

# variaciones tabu super culs

In [5]:
experimentos_TABU_optimizados = []

for n in range(0, df_grafos_propios.shape[0]):
    fila_n = df_grafos_propios.iloc[n];
    for algo in algoritmos_tabu_optimizado:
        experimentos_TABU_optimizados.append([fila_n["dataset"], fila_n["n"], fila_n["m"], algo, fila_n["archivo"]]);

In [6]:
columnas = ["dataset", "n", "m", "metodo", "tiempo", "costo"];
filas = [];
numero = 1
T = 3 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos_TABU_optimizados:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos_TABU_optimizados)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    costos  = []
    tiempos = []
    for i in range(0, T):
        resultado = correr_experimento(experimento[3], experimento[4])
        algoritmo = experimento[4]
        costos.append(resultado[2])
        tiempos.append(resultado[0]);
    costo = min(costos)
    tiempo = np.mean(tiempos);
    filas.append([experimento[0], experimento[1], experimento[2], experimento[3], tiempo,costo ]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultado_tabu_mejor_configuracion_ins.csv", index=False, header=True);

'Experimento: 200/200'

'TABU_ESTRUCTURA'

'TABU_ESTRUCTURA'

'TABU_ESTRUCTURA'